## Old Hope

In [1]:
import asyncio
from playwright.async_api import async_playwright
from time import sleep

TIMED_SHUTDOWN: bool = True

In [2]:
def save_names(f_names, filename):
    with open(filename, "w", encoding="utf-8") as f:
        for f_name in f_names:
            f.write(f_name)


# TODO:
# figure out how to get rid of "Verified" when writing to file
async def get_names(page, limit, filename):
    await page.keyboard.press("Tab")
    await page.keyboard.press("Tab")
    await page.keyboard.press("Tab")
    await page.keyboard.press("Tab")

    f_names = set()
    same_length_count: int = 0
    f_names_old_length: int = 0

    for i in range(limit):
        for _ in range(20):
            await page.keyboard.press("ArrowDown")
            sleep(1)

        if i % 10 == 0:
            tmp_name_list = await page.locator("role=link").all_inner_texts()
            f_names.update(tmp_name_list)
            if len(f_names) >= limit:
                break

            if f_names_old_length == len(f_names):
                same_length_count += 1
                print(f"stuck count: {same_length_count}")
            else:
                same_length_count = 0
                f_names_old_length = len(f_names)
                print(f"current length: {len(f_names)}")

            if same_length_count >= 3:
                break

        sleep(2)

    tmp_name_list = await page.locator("role=link").all_inner_texts()
    f_names.update(tmp_name_list)

    for name in ("", "primetimetank_", "explore", "Verified"):
        try:
            f_names.remove(name)
        except:
            continue

    with open(filename, "w", encoding="utf-8") as f:
        for f_name in f_names:
            f.write(f"https://www.instagram.com/{f_name}\n")

    f_names.clear()

    with open(filename, "r", encoding="utf-8") as f:
        for f_name in f.readlines():
            if f_name != "Verified":
                f_names.add(f_name)

    save_names(f_names=f_names, filename=filename)

    return f_names


async def run(playwright):
    firefox = playwright.firefox
    browser = await firefox.launch(headless=False)
    context = await browser.new_context(storage_state="instagram.json")
    page = await context.new_page()
    url = "https://www.instagram.com/primetimetank_"
    await page.goto(url)
    sleep(10)

    followers_amount = await page.locator("text=followers").all_inner_texts()
    followers_amount = int(followers_amount[0].split(" ")[0].replace(",", ""))

    following_amount = await page.locator("text=following").all_inner_texts()
    following_amount = int(following_amount[0].split(" ")[0].replace(",", ""))

    await page.goto(f"{url}/followers")
    sleep(4)

    followers_names = await get_names(
        page=page,
        limit=followers_amount,
        filename="followers_links.txt",
    )
    print(f"Followers: {len(followers_names)}")

    await page.goto(f"{url}/following")
    sleep(4)
    following_names = await get_names(
        page=page,
        limit=following_amount,
        filename="following_links.txt",
    )
    print(f"Following: {len(following_names)}")

    not_following_me_back = following_names.difference(followers_names)
    im_not_following_back = followers_names.difference(following_names)

    print(f"Not following me back: {len(not_following_me_back)}")
    save_names(f_names=not_following_me_back, filename="not_following_me_back.txt")

    print(f"I'm not following back: {len(im_not_following_back)}")
    save_names(f_names=im_not_following_back, filename="im_not_following_back.txt")

    await context.close()
    await browser.close()


async def main():
    async with async_playwright() as playwright:
        await run(playwright)

In [3]:
# async with async_playwright() as playwright:
#     firefox = playwright.firefox
#     browser = await firefox.launch(headless=False)
#     context = await browser.new_context(storage_state="instagram.json")
#     page = await context.new_page()
#     url = "https://www.instagram.com/primetimetank_"
#     await page.goto(url)

In [4]:
# firefox

In [6]:
pr = await async_playwright().start()
firefox = pr.firefox
browser = await firefox.launch(headless=False)
context = await browser.new_context(storage_state="instagram.json")
page = await context.new_page()
url = "https://www.instagram.com/primetimetank_"
await page.goto(url)
sleep(15)

In [7]:
cookies = await context.cookies()
ds_user_id:str = ""

In [8]:
# TODO: left off here
for cookie in cookies:
    if cookie["name"] == "ds_user_id":
        print(cookie)
        print(cookie["name"],":",cookie["value"])
        ds_user_id = cookie["value"]
        break

{'name': 'ds_user_id', 'value': '462306021', 'domain': '.instagram.com', 'path': '/', 'expires': 1735142663, 'httpOnly': False, 'secure': True, 'sameSite': 'None'}
ds_user_id : 462306021


In [69]:
followers_amount = await page.locator("text=followers").all_inner_texts()
followers_amount = int(followers_amount[0].split(" ")[0].replace(",", ""))

following_amount = await page.locator("text=following").all_inner_texts()
following_amount = int(following_amount[0].split(" ")[0].replace(",", ""))

In [70]:
followers_amount, following_amount

(1461, 976)

In [8]:
await page.goto(f"{url}/followers")
sleep(15)

<Response url='https://www.instagram.com/primetimetank_/followers' request=<Request url='https://www.instagram.com/primetimetank_/followers' method='GET'>>

In [15]:
await page.locator("text=followers").click()
sleep(5)

In [16]:
followers_names = await get_names(
    page=page,
    # text="Followers",
    # nth=-2,
    limit=followers_amount,
    filename="followers_links.txt",
)
print(f"Followers: {len(followers_names)}")

current length: 21
current length: 172
current length: 328
current length: 474
current length: 621
current length: 773
current length: 920
current length: 1070
current length: 1229
current length: 1375
current length: 1470
stuck count: 1
stuck count: 2
stuck count: 3
Followers: 1469


In [17]:
await page.goto(f"{url}/following")
sleep(15)

In [18]:
await page.locator("text=following").click()
sleep(5)

In [19]:
following_names = await get_names(
    page=page,
    # text="Following",
    # nth=3,
    limit=following_amount,
    filename="following_links.txt",
)
print(f"Following: {len(following_names)}")

current length: 25
current length: 181
current length: 325
current length: 469
current length: 625
current length: 769
current length: 917
current length: 970
stuck count: 1
stuck count: 2
stuck count: 3
Following: 969


In [20]:
not_following_me_back = following_names.difference(followers_names)
im_not_following_back = followers_names.difference(following_names)

print(f"Not following me back: {len(not_following_me_back)}")
save_names(f_names=not_following_me_back, filename="not_following_me_back.txt")

print(f"I'm not following back: {len(im_not_following_back)}")
save_names(f_names=im_not_following_back, filename="im_not_following_back.txt")

Not following me back: 71
I'm not following back: 571


In [38]:
await context.close()
await browser.close()

In [24]:
if TIMED_SHUTDOWN:
    !shutdown.exe -s -t 5

## A New Hope

### Imports

In [16]:
import asyncio
from typing import List, Dict
from playwright.async_api import async_playwright
from time import sleep
# from bs4 import BeautifulSoup
import requests
import json
from pathlib import Path

### Initialization

In [31]:
COUNT = 100
max_id:str = ""
x_ig_app_id:str = ""
x_asbd_id:str = ""
instagram_users = {"followers": [], "following": []}

In [32]:
pr = await async_playwright().start()
firefox = pr.firefox
browser = await firefox.launch(headless=False)
context = await browser.new_context(storage_state="instagram.json")
page = await context.new_page()
url = "https://www.instagram.com/primetimetank_"
await page.goto(url)
cookies:List[Dict[str,str]] = await context.cookies()

### Get Cookies

In [33]:
ds_user_id:str = ""

for cookie in cookies:
    # print(f"{cookie['name']:<20}:\t\t{cookie['value'][:25]}")
    if cookie["name"] == "ds_user_id":
        # print(cookie)
        # print(cookie["name"],":",cookie["value"])
        ds_user_id = cookie["value"]

# Reformat the cookies
cookie_dict:Dict[str,str] = {cookie["name"]: cookie["value"] for cookie in cookies}

### Get Expected User Amounts

In [34]:
followers_amount = await page.locator("text=followers").all_inner_texts()
followers_amount = int(followers_amount[0].split(" ")[0].replace(",", ""))

following_amount = await page.locator("text=following").all_inner_texts()
following_amount = int(following_amount[0].split(" ")[0].replace(",", ""))

followers_amount, following_amount

(1460, 976)

In [ ]:
# for cookie in cookies:
#     print(f"{cookie['name']:<20}:\t\t{cookie['value'][:50]}")

In [37]:
# type(cookies)
# cookies #type: list
# cookie_dict = {cookie["name"]: cookie["value"] for cookie in cookies}
# cookie_dict

### Get `X-IG-App-ID` header for request

In [35]:
# TODO: move this before first request
a_handle = await page.evaluate_handle("document.body")
result_handle = await page.evaluate_handle("body => body.innerHTML", a_handle)
s = await result_handle.json_value()
# print(await result_handle.json_value())
await result_handle.dispose()

In [36]:
if "X-IG-App-ID" in s:
    starting_index:int = s.index("X-IG-App-ID")
    end_index:int = s[starting_index:starting_index+50].index("}")
    x_ig_app_id = str(int(s[starting_index:starting_index+end_index].split(",")[0].split(":")[1].replace('"',"").replace('"',"")))
    print(x_ig_app_id)

936619743392459


### Send Request

In [37]:
for key in instagram_users.keys():
# for key in ["following"]:
    user_count:int = 0
    expected_max_users:int = -1
    stuck_count:int = 0
    # has_more:bool = True
    
    if key=="followers":
        expected_max_users = followers_amount
    elif key == "following":
        expected_max_users = following_amount
    else:
        raise Exception("Error: Invalid key (should be 'followers' or 'following'")

    # TODO: think about max_id as well
    while all([user_count < expected_max_users, stuck_count < 5]):
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:130.0) Gecko/20100101 Firefox/130.0',
            'Accept': '*/*',
            'Accept-Language': 'en-US,en;q=0.5',
            # 'Accept-Encoding': 'gzip, deflate, br, zstd',
            
            # 'X-CSRFToken': 'pDsZRt6b2nEbFGNml8lRYqaGoPdSrYaI',
            # 'X-IG-App-ID': '936619743392459',
            'X-IG-App-ID': x_ig_app_id,
            # 'X-ASBD-ID': '129477',
            # 'X-IG-WWW-Claim': 'hmac.AR1Jpv_YBocGDWkC94CGCH0NmZzE5k5vqnemRiLTQjt7PrjC',
            
            'X-Requested-With': 'XMLHttpRequest',
            'Alt-Used': 'www.instagram.com',
            'Connection': 'keep-alive',
            'Referer': f'https://www.instagram.com/primetimetank_/{key}/',
            # 'Cookie': 'csrftoken=pDsZRt6b2nEbFGNml8lRYqaGoPdSrYaI; mid=ZtZ85gALAAESN8NIkokirz8K-FF3; datr=5nzWZh52P16wwD-Pm-yDAswC; ig_did=64F3AF16-D093-4BA8-B45C-DDF8F6D0858E; ig_nrcb=1; ds_user_id=462306021; sessionid=462306021%3A9FVsXC1BtaURuO%3A11%3AAYez3fX-ylI8-Zbhaoig6lzg4VHimLpVoVvUAeFZEw; ps_l=1; ps_n=1; shbid="4118\\054462306021\\0541758830867:01f7c71df583cfeb66704de42335bceed6e6c2bab534ac91b5a2f741a3a50192f448f849"; shbts="1727294867\\054462306021\\0541758830867:01f7ec62e942af9376c0501508a0e29def34a6e9ef706c865ce43ddf882aa21a830a968c"; rur="CCO\\054462306021\\0541758830926:01f78d2e42983910bd75fe6b668be8feb8b766f75ddfc70cbe610f8ed5d797a23dda64e6"; wd=1920x509',
            'Sec-Fetch-Dest': 'empty',
            'Sec-Fetch-Mode': 'cors',
            'Sec-Fetch-Site': 'same-origin',
            # Requests doesn't support trailers
            # 'TE': 'trailers',
        }
        
        params = {
            # 'count': '100',
            'count': COUNT,
            # 'max_id': 'QVFDczNtemVwQ2oydFJVemJ3UUxWaGI1N3RIT0x1cjFZbGlMejhkMGcxdnVyOE04ZVpPQmt5VUVUTkk1Vm5CZkRvd0l3clJyVm5YMDJjT1g0Tmo0WTVPag==',
            'max_id': max_id,
            'search_surface': 'follow_list_page',
        }

        try:
            response = requests.get(
                # 'https://www.instagram.com/api/v1/friendships/462306021/followers/',
                f'https://www.instagram.com/api/v1/friendships/{ds_user_id}/{key}/',
                params=params,
                cookies=cookie_dict,
                headers=headers,
            )


            response_dict = response.json()            

            instagram_users[key] += response_dict["users"]
            user_count = len(instagram_users[key])
            
            with open("instagram_users.json","w",encoding="utf-8") as f:
                json.dump(instagram_users, f, indent=4)

            print(f"Updated instagram_users for {key} (current length: {user_count})")

            # prepare for next loop iteration
            # has_more = response_dict["has_more"]
            max_id = response_dict["next_max_id"]
            stuck_count = 0
            
        except Exception as e:
            # print(e.with_traceback())
            print(e)
            stuck_count += 1
            print(f"Got stuck: {stuck_count}")
        finally:
            sleep(5)
        

Updated instagram_users for followers (current length: 21)
Updated instagram_users for followers (current length: 44)
Updated instagram_users for followers (current length: 67)
Updated instagram_users for followers (current length: 90)
Updated instagram_users for followers (current length: 112)
Updated instagram_users for followers (current length: 137)
Updated instagram_users for followers (current length: 155)
Updated instagram_users for followers (current length: 177)
Updated instagram_users for followers (current length: 196)
Updated instagram_users for followers (current length: 220)
Updated instagram_users for followers (current length: 244)
Updated instagram_users for followers (current length: 268)
Updated instagram_users for followers (current length: 293)
Updated instagram_users for followers (current length: 312)
Updated instagram_users for followers (current length: 333)
Updated instagram_users for followers (current length: 354)
Updated instagram_users for followers (curre

### Get Intersections

In [26]:
response.json().keys()

dict_keys(['users', 'big_list', 'page_size', 'next_max_id', 'has_more', 'should_limit_list_of_followers', 'use_clickable_see_more', 'show_spam_follow_request_tab', 'status'])

In [27]:
response.json()["has_more"]

False

### Analyze Response

In [4]:
with open("instagram_users.json","r",encoding="utf-8") as f:
    user_dict = json.load(f)

In [5]:
len(user_dict["followers"]), len(user_dict["following"])

(1461, 976)

In [9]:
not_following_me_back:List[dict] = []

In [ ]:
for instagram_user_im_following in user_dict["following"]:
    user_name_im_following = instagram_user_im_following["username"]
    # print(user_name_im_following, end=" ")
    is_following_me_back:bool = False
    for instagram_follower in user_dict["followers"]:
        if user_name_im_following == instagram_follower["username"]:
            is_following_me_back = True
            break
    if not is_following_me_back:
        not_following_me_back.append(instagram_user_im_following)
        # print(f"{user_name_im_following} is not following me back :(")
    # print("is not following me back :(") if not is_following_me_back else print("is following me!")

len(not_following_me_back)

In [14]:
im_not_following_back:List[dict] = []

In [ ]:
for instagram_user_following_me in user_dict["followers"]:
    user_name_following_me = instagram_user_following_me["username"]
    im_following_back:bool = False
    for instagram_following in user_dict["following"]:
        if user_name_following_me == instagram_following["username"]:
            im_following_back = True
            break
    if not im_following_back:
        im_not_following_back.append(instagram_user_following_me)
        # print(f"{user_name_following_me} is someone I'm not following back")

len(im_not_following_back)

In [17]:
instagram_stats_dir = Path(Path.cwd(), "statistics")
instagram_stats_dir.mkdir(exist_ok=True, parents=True)

In [18]:
with open(Path(instagram_stats_dir, "not_following_me_back.txt"),"w",encoding="utf-8") as f:
    for instagram_user in not_following_me_back:
        user_name = instagram_user["username"]
        f.write(f"https://www.instagram.com/{user_name}\n")

In [19]:
with open(Path(instagram_stats_dir, "im_not_following_back.txt"),"w",encoding="utf-8") as f:
    for instagram_user in im_not_following_back:
        user_name = instagram_user["username"]
        f.write(f"https://www.instagram.com/{user_name}\n")